# Bostrom supply simulation
---

## Goals

Optimize parametrs for launching Bostrom.

An idea is to model value of BOOT through undestanding of established network effects in Ethereum and Fantom.
Then we can forecast claim dynamics and address growth based on aproximated network effects. Asuming some demand for cyberlinks based on address growth we can adjust supply of cyberlinks so that V price could grow. Given model also allow to define inflation parametrs of BOOT to optimize investments into the hardware infrustructure.


## Understanding network effects

Modeling price of BOOT as a function of usage weighted on network effects.  

Simulate agent base:
- startAgentCount = 100000
- agentsCount:

\begin{gather}
\\
agentsCount = 8.97*days^2 + 105*days + 100000\\
\\
\end{gather}

Simulate capitalisation:
- StartingAgentValue = 1 ETH
- CapitalisationPerUser:

\begin{gather}
\\
CapitalisationPerUser = 5011.9 * agentsCount^{-0,74} \\
\\
\end{gather}



Simulate gift claim dynamics.  
Parameters to define:
- MinGiftedAgents  
Assumtions:
- ClaimRate (0.1% per day)


## Predicting V demand

Simulate cyberlinks usage.  

- Name (ens index)
- Following (proportion of agents)
- Extra 
- Guaranted links by the founding team

Base estimate of cyberlink per agent:

\begin{gather}
\\
CyberlinksPerAgent = 9 \cdot agentsCount^{-0.27} \\
\\
\end{gather}



## Adjusting V supply

Model minting properties of V (and A = V) for planing of GPU storage and maximisation of price.  
Parameters to define for A and V:
- InvestmintPeriod
- InvestmintAmount
- MintrateFunction
- MaxInvestmintSchedule  
Asumptions:
- LockRate (70% of bonded)

## Investments into hardware

Simulate ability of heroes to invest into infrastructure depending on different market conditions.  
Parameters to define for BOOT:  
- InflationRateChange
- InflationMax
- InflationMin
- GoalBonded  
Asumptions: 
- BondedRate (70% of supply)

The formula used in excel model:

\begin{gather}
\\
supply = supplyBase \cdot ( 1 + \frac{ 0.12}{1 + \frac{days}{365 \cdot 2}})^{\frac{days}{365}} \\
\\
\end{gather}

## Further work

Model could work as prediction governance tool for working network based on acutal measurements.

## Legend

There are three network tokens:

* F - Unclaimed network token
* T - Liquid network token
* L - Frozen network token


and two resource tokens:

* A - Amper token
* V - Volt token

The initial state of all tokens should be defined below

* 1 **timestep** == 1 time unit

## Claim function

The function of claim frozen tokens is:

\begin{gather}
\\
f(x) = 7 \cdot 10^{14} - \frac{5.6 \cdot 10^{14}}{3}x \\
\\
\end{gather}

<img src='images/claim.png' />

where **x** is year. This function means 80% of frozen tokens are going to be claimed in 3 years linearly by ~187 TeraTokens per year

## Vesting and Unvesting

The vesting function is defined as the amount of locking tokens in the time unit assumed by all liquid tokens must be locked in the lock timeframe. 

The vesting function is defined as the amount of unlocking tokens in the time unit assumed by all locked tokens must be unlocked in the unlock timeframe. 

## Resource tokens

Amperes and Volts mints by the following formula:

\begin{gather}
\\
f(x) = \frac{x \cdot \frac{maxLockTime}{cycle}}{initPrice} \cdot MR^r \\
\\
\end{gather}

where: 
* **x** is number of vested tokens
* **lockTime** is the maximum time for locking tokens
* **cycle** is amount of blocks for rank and entropy calculation
* **initPrice** is the initial price of the minting
* **MR_r** is mint rate [100, 1]. It's different for all energy tokens



## Mint Rate (Amperes)

Mint rate is multiple coefficient for minting Amper tokens

It is halving every year

\begin{gather}
\\
MR_a(x) = \frac{initMR^a}{2^{\lfloor{x}\rfloor}}\\
\\
\end{gather}

where: 
* **x** is year
* **initMRa** is the initial value of MRa

f.e.

<img src='images/mr_a.png' />


## Mint Rate (Voltes)

Mint rate is multiple coefficient for minting Volt tokens

It is halving every amount of supply defined

\begin{gather}
\\
MR_v(x) = \frac{initMR^v}{2^{\lfloor{x}\rfloor}}\\
\\
\end{gather}

where: 
* **x** is defined supply
* **initMRv** is the initial value of MRv


## Inflation

The inflation function depends on the ratio between vested tokens and tokens supply. It the same as in cosmos-based networks. 

## Supply

Supply is the sum of liquid, vested and frozen tokens in each block.


## Utils

The function of using volts for cyberlinking. By the simulation the probability in a timestamp [0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.21, 0.34] .


## Assumptions

1. All agents lock tokens for the maximum available period defined in params for simulating
2. All agents mint maximum **A** and **V** tokens in 50/50 ratio

## Mathematical Specification

### Differential Equations

* T - liquid network token
* L - locked(vested) network token
* F - not claimed(frozen) network token
* IRC_u - inflation rate change per time unit
* S - total network tokens supply
* I - inflation per time unit in network tokens
* Provision - per time unit token provision
* A - Amper token
* V - Volt token
* CL - cyberlinks

\begin{gather}
\\
T_u = T_{u-1} + {\Delta T} \tag{1} \\
L_u = V_{u-1} + {\Delta V} \tag{2} \\
F_u = F_{u-1} + {\Delta F} \tag{3} \\
A_u = A_{u-1} + {\Delta A} \tag{4} \\
V_u = V_{u-1} + {\Delta V} \tag{5} \\ \\
MR^{a}_{u} = {\Delta MR^a} \tag{6} \\ \\
MR^{v}_{u} = {\Delta MR^v} \tag{7} \\ \\
CL_{u} = CL_{u-1} + {\Delta CL} \tag{8} \\ \\
S_u = T_u + L_u + F_u \tag{9} \\ 
\\
\end{gather}

where the rate of change ($\Delta$) is:
\begin{gather}
\\
{\Delta F} = - \frac{5.6 \cdot 10^{14}}{3 \cdot unitsPerYear} \tag{10} \\ \\
{\Delta L} = \frac{T_{u-1}}{\frac{unitsPerYear}{12} \cdot vestingSpeed} - {\Delta U} \tag{11} \\ \\
{\Delta T} = - {\Delta F} - {\Delta L} + I_{u-1} + {\Delta U}  \tag{12} \\ \\
{\Delta A} = \lfloor{\lfloor{\frac{\frac{1}{2} \cdot \Delta L}{initPrice} \cdot \frac{vestringTime_{max}}{baseVestingTime}}\rfloor \cdot MR^a}\rfloor  \tag{13} \\ \\
{\Delta V} = \lfloor{\lfloor{\frac{\frac{1}{2} \cdot \Delta L}{initPrice} \cdot \frac{vestringTime_{max}}{baseVestingTime}}\rfloor \cdot MR^v}\rfloor  \tag{14} \\ \\
{\Delta U} = \frac{V_{u-1}}{\frac{unitsPerYear}{12} \cdot unvestingSpeed} \tag{15} \\ \\
{\Delta MR^a} = \frac{MR^a_{init}}{2^{\lfloor{\frac{u}{unitsPerYear}}\rfloor}} \tag{16} \\ \\
{\Delta MR^v} = \frac{MR^v_{init}}{2^{\lfloor{\frac{V}{voltHalving}}\rfloor}} \tag{17} \\ \\
{\Delta CL} = V \cdot Ut(u) \tag{18} \\ \\
\\
\end{gather}

where:
\begin{gather}
\\
I_{u-1} = \frac{S_{u-1} \cdot IRC_{u-1}}{unitsPerYear} \tag{19} \\ \\
IRC_u = \frac{\left(1 - \frac{vestedRatio_{u-1}}{goalVested}\right) \cdot inflationRateChange}{unitsPerYear} \tag{20} \\ \\
{vestingTime_{max} = initVestingTime_{max} \cdot 2^{\lfloor{\frac{u}{unitsPerYear}}\rfloor}} \tag{21} \\ \\
Ut(u) = [0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.21, 0.34] \tag{22} 
\\
\\
\end{gather}

## A/B simulations params


| Param name | Simulation A | Simulation B|
|:---|---:|---:|
|Liquid Tokens (T)|200,000,000,000,000|200,000,000,000,000|
|Frozen Tokens (F)|700,000,000,000,000|700,000,000,000,000|
|Vested Tokens (L)|100,000,000,000,000|100,000,000,000,000|
|Initial inflation |5.00|5.00|
|Amper Mint Rate init|50.00|50.00|
|Volt Mint Rate init|100.00|100.00|
|Inflation rate change|13.00|13.00|
|Inflation min|1.00|1.00|
|Inflation max|15.00|15.00|
|Goal vested|80.00|80.00|
|Volt halving|1,000,000.00|1,000,000.00|
|Resource token init price|100,000,000.00|100,000,000.00|

In [ ]:
import time

# Standard libraries: https://docs.python.org/3/library/
import math

# Analysis and plotting modules
import pandas as pd

## Initial state

In [ ]:
simPeriod = 10          # amount of years for simulating
unitsPerYear = 365        # units per year(days)

In [ ]:
initial_state_a = {
    'T': 200_000_000_000_000,
    'F': 700_000_000_000_000,
    'L': 100_000_000_000_000,
    'A': 0,
    'V': 0,
    'd_u': 0,
    'I_r': 0.05,
    'd_l': 0,
    'MRa': 50,
    'MRv': 100,
    'CL': 0,
    'maxVestingTime': math.floor(unitsPerYear / 4)
}

In [ ]:
# cadCAD configuration modules
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment

# cadCAD simulation engine modules
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

from collections import Counter
from cadCAD import configs

# custom functions modules

from utils.psub import partial_state_update_blocks

## Params for simulating

In [ ]:
import itertools

vestingSpeed = [3, 6, 12, 18, 24, 30, 36]
unvestingSpeed = [72, 60, 48, 36, 30, 24]
cyberlinks_util = [0.01, 0.02, 0.03, 0.05, 0.08, 0.13, 0.21, 0.34]

parameter_sweep = list(itertools.product(vestingSpeed, unvestingSpeed, cyberlinks_util))

vestingSpeed = [x[0] for x in parameter_sweep]
unvestingSpeed = [x[1] for x in parameter_sweep]
cyberlinks_util = [x[2] for x in parameter_sweep]

In [ ]:
system_params_a = {
    'inflationRateChange': [0.13], # maximum inflation rate change per year
    'inflationMin': [0.01], # minimum percent of the inflation
    'inflationMax': [0.15], # maximum percent of the inflation
    'goalVested': [0.80], # desireable ratio between vested tokens and tokens supply
    'unitsPerYear': [unitsPerYear], # units per year(minutes)
    'vestingSpeed': vestingSpeed, # amount of months to vest all liquid tokens
    'unvestingSpeed': unvestingSpeed, # amount of months to uninvest all vested tokens
    'voltHalving': [10_000_000], # 10 MegaVolt
    'cyberlinks_util': cyberlinks_util,
    'MRa': [50], # initial mint rate for Amperes
    'MRv': [100], # initial mint rate for Voltes
    'initPrice': [10_000_000], # initial cost for resource token in 1 cycle
    'baseVestingTime': [math.floor(unitsPerYear / 4)]
}

In [ ]:
del configs[:]
experiment = Experiment()

sim_config = config_sim({
    'N': 1,
    'T': range(int(math.ceil(simPeriod * unitsPerYear))),
    'M': system_params_a
})

experiment.append_configs(
    initial_state = initial_state_a,
    partial_state_update_blocks = partial_state_update_blocks,
    sim_configs = sim_config
)

In [ ]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()

In [ ]:
start_time = time.time()
simulation_result = pd.DataFrame(raw_result)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()

In [ ]:
df = simulation_result.copy()
df['Supply'] = df['T'] + df['F'] + df['L']
df['AmPrice'] = df['A'] / df['V']
df

In [ ]:
df

In [ ]:
# save local model file for power bi
# import pyarrow as pa
# import pyarrow.parquet as pq
# table = pa.Table.from_pandas(df)
# pq.write_table(table, 'simulation.parquet')

In [ ]:
from utils.plots import linear_plot, scatter_plot, df_preparator

In [ ]:
plot_df = df_preparator(df)

## Inflation

In [ ]:
scatter_plot(plot_df, "I_r")

## Supply

In [ ]:
scatter_plot(plot_df, 'Supply')

## Vested

In [ ]:
scatter_plot(plot_df, 'L')

## Ampler tokens supply

In [ ]:
scatter_plot(plot_df, 'A')

## Volt tokens supply

In [ ]:
scatter_plot(plot_df, 'V')

## Mint Rate Volter-Amper

In [ ]:
linear_plot(plot_df, ['MRa', 'MRv'])

## Max vesting time

In [ ]:
linear_plot(plot_df, 'maxVestingTime')

## Ratio between amper supply and volt supply

In [ ]:
scatter_plot(plot_df, 'AmPrice')

## Cyberlinks

In [ ]:
scatter_plot(plot_df, 'CL')

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))